# Numerical Methods

## Lecture 7: Numerical Linear Algebra III

### Exercise solutions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import scipy.linalg as sl

### <span style="color:blue">Exercise 7.1: matrix norms</span>

Write some code to explicitly compute the two matrix norms defined mathematically above and compare against the values found above using in-built scipy functions.

Based on the above code and comments, what is the mathematical definition of the 1-norm and the 2-norm?


In [4]:

def frob(A):
    m, n = A.shape
    squsum = 0.
    for i in range(m):
        for j in range(n):
            squsum += A[i,j]**2
    return np.sqrt(squsum)


def mars(A):
    m, n = A.shape
    maxarsum = 0.
    for i in range(m):
        arsum = np.sum(np.abs(A[i]))
        maxarsum = arsum if arsum > maxarsum else maxarsum
    return maxarsum


A = np.array([[10., 2., 1.],
                 [6., 5., 4.],
                 [1., 4., 7.]])
frob(A) == sl.norm(A,'fro') and mars(A) == sl.norm(A,np.inf)

True

### <span style="color:blue">Exercise 7.2: Ill-conditioned matrix</span>

Consider a range of small values for $\epsilon$ and calculate the matrix determinant and condition number.

In [13]:
A = np.array([[2.,1.],
                 [2.,1.]])
b = np.array([3.,0.])
print(sl.det(A), 'singular')

for i in range(3):
    A[1,1] += 0.001
    print(sl.det(A), sl.inv(A) @ b)

0.0 singular
0.0019999999999997797 [ 1501.5 -3000. ]
0.0039999999999995595 [  751.5 -1500. ]
0.005999999999999339 [  501.5 -1000. ]


### <span style="color:blue">Exercise 7.3: Implement Gauss-Seidel's method.</span>

Generalise the Jacobi code to solve the matrix problem using Gauss-Seidel's method.

In [12]:
def gauss_seidel(A, b, maxit=500):
    m, n = A.shape
    x = np.zeros_like(b)
    for k in range(maxit):
        for i in range(m):
            x[i] = 1/A[i,i] * (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,i+1:], x[i+1:]))
    return x


A = np.array([[10., 2., 3., 5.],
                 [1., 14., 6., 2.],
                 [-1., 4., 16.,-4],
                 [5. ,4. ,3. ,11.]])
b = np.array([1., 2., 3., 4.])

# check gauss_seidel solution agrees with multiplying through by the inverse matrix
np.allclose(sl.inv(A)@b, gauss_seidel(A, b))

True